In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np
import os
from google.colab import drive
from collections import Counter
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/TR_DIMA/Logit_compensation/')
from logitadjust import LogitAdjust



# Path to the dataset and model
path_train_dataset = "/content/drive/MyDrive/TR_DIMA/training_set_reduit"

# Transformations for the dataset
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Data augmentation

augmented_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.GaussianBlur(kernel_size=9, sigma=(0.01, 5)),
        transforms.RandomResizedCrop(size=224, scale=(0.7, 1.0), ratio = (0.2,5)),
        transforms.ToTensor(),
    ])









Mounted at /content/drive


In [ ]:
# Load the training dataset

dataset_init = datasets.ImageFolder(root=path_train_dataset, transform=augmented_transform)



train_size = int(0.8 * len(dataset_init))
valid_size = len(dataset_init) - train_size

train_dataset, valid_dataset = random_split(dataset_init, [train_size, valid_size])

# Create DataLoaders for training and validation datasets

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=12, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)

valid_classes = dataset_init.classes

In [ ]:
# Number of elements in each class

num_all = Counter(dataset_init.targets)
distrib = [num_all[i] for i in range(0,13)]

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(valid_classes))

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 197MB/s]


In [ ]:
# Training function for the first step of decoupling framework

num_epochs = 100
patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)

    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        train_bar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)

    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_bar.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_loader.dataset)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/TR_DIMA/Entrainement/best_model_pre_classifier_training.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping déclenché.")
            break



KeyboardInterrupt: 

On charge l'ancien modèle et on créé un nouveau sampler equilibré

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(valid_classes))
model.to(device)


# Charger le meilleur modèle
model.load_state_dict(torch.load("/content/drive/MyDrive/TR_DIMA/Entrainement/best_model_pre_classifier_training.pth"))

#Second stage of training

train_indices = train_dataset.indices
train_labels = [dataset_init.samples[i][1] for i in train_indices]

class_counts = Counter(train_labels)
class_weights = {label: 1.0 / count for label, count in class_counts.items()}
sample_weights = [class_weights[label] for label in train_labels]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples= 4*len(sample_weights), replacement=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[0.012046444248795023, 0.006100056303779527, 0.006100056303779527, 0.03641785203646149, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.012046444248795023, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.012046444248795023, 0.012046444248795023, 0.006100056303779527, 0.006100056303779527, 0.012046444248795023, 0.012046444248795023, 0.012046444248795023, 0.006100056303779527, 0.012046444248795023, 0.09853292781642932, 0.006100056303779527, 0.012046444248795023, 0.012046444248795023, 0.006100056303779527, 0.006100056303779527, 0.012046444248795023, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.006100056303779527, 0.012046444248795023, 0.006100056303779527, 0.006100056

In [ ]:
# Create a new DataLoader with the sampler

train_loader = DataLoader(train_dataset, batch_size=128, sampler=sampler, num_workers=16, pin_memory=True)

# Reset the model's fully connected layer for the second stage of training

model.fc.reset_parameters()


# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

# Define the loss function and optimizer for the second stage
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-5, weight_decay=1e-4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
num_epochs = 100
patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)

    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        train_bar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)

    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_bar.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_loader.dataset)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/TR_DIMA/Entrainement/best_model_post_classifier_training.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping déclenché.")
            break




Epoch 1: Train Loss = 3.9165, Val Loss = 0.2556


Epoch 2: Train Loss = 1.5204, Val Loss = 0.1565


Epoch 3: Train Loss = 1.1512, Val Loss = 0.1340


Epoch 4: Train Loss = 1.0086, Val Loss = 0.1194


Epoch 5: Train Loss = 0.9350, Val Loss = 0.1137


...

Epoch 16: Train Loss = 0.7702, Val Loss = 0.1073


Epoch 17: Train Loss = 0.7766, Val Loss = 0.1090


Epoch 18: Train Loss = 0.7648, Val Loss = 0.1036


Epoch 19: Train Loss = 0.7559, Val Loss = 0.1057


Epoch 20: Train Loss = 0.7430, Val Loss = 0.1008
Early stopping déclenché.
